This is a live notebook with experimental code to develop functionality for grouping and cross validation.

In [16]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import itertools


from sklearn.model_selection import GroupKFold

import matplotlib.pyplot as plt
import pandas as pd

import os
print(os.listdir("."))


import re
res_digit = r'[0-9]'

['debug_windowing_data.ipynb', 'feature_vs_condition_correlations.png', 'testing_mlpwrapper.ipynb', 'group_standardisation_of_features.ipynb', 'logs', 'feature_pairwise_plotting.ipynb', 'analyse_results.ipynb', 'gait_data_features.ipynb', 'feature_feature_correlations.ipynb', 'feature_development.ipynb', 'data', 'test_sample_entropy_implementations.ipynb', 'merging_dataframes.ipynb', 'data_and_processing_description.ipynb', 'feature_comparator_correlations.ipynb', 'featured_prediction_random_forest.ipynb', 'window_timings_and_IBI.ipynb', 'gait_data_exploration.ipynb', 'processing_status.ipynb', '.ipynb_checkpoints', 'loading_e4_data.ipynb', 'data_investigation_scratch.ipynb', 'simply_load_and_inspect_data.ipynb', 'grouping_and_crossvalidation.ipynb', 'sample-entropy-numba-impl.ipynb', 'featured_prediction.ipynb']


In [2]:
# This is a hack to make the library in the parent folder available for imoprts
# A better solution is by np8 here:
# https://stackoverflow.com/questions/714063/importing-modules-from-parent-folder
import sys
import os
import inspect

thisdir = sys.path[0]
print(f"thisdir = {thisdir}")
parentdir = os.path.dirname(thisdir)
#print(f"parentdir = {parentdir}")
if not parentdir in sys.path:
    print("Adding parent directory to python path")
    sys.path.insert(1, parentdir)
else:
    print("Skipping adding parent direct to path (there already)")

print(f"sys.path =\n{sys.path}")


thisdir = /home/luke/git/external/predicament/notebooks
Adding parent directory to python path
sys.path =
['/home/luke/git/external/predicament/notebooks', '/home/luke/git/external/predicament', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/luke/.local/lib/python3.10/site-packages', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/lib/python3.10/dist-packages']


In [3]:
## ensure relative path to data directory is sound
# for the notebook we need to modify the BASE_DATA_FOLDER
import os 
os.environ['PREDICAMENT_DATA_DIR'] =  '../data'

In [10]:
from predicament.utils.file_utils import load_featured_dataframe_and_config
from predicament.evaluation.grouping import get_group_assignments
from predicament.evaluation.staging import get_design_matrix


## Load data

In [17]:
subdir = 'E4_10secs'
df, config = load_featured_dataframe_and_config(
    subdir)

n_channels = config['LOAD']['n_channels']
channels = config['LOAD']['channels']
participant_list = config['LOAD']['participant_list']
sample_rate = config['LOAD']['sample_rate']
window_size = config['WINDOWED']['window_size']
feature_set = config['FEATURED']['feature_set']
time = window_size/sample_rate
print(f"sample_rate: {sample_rate}, n_samples = {window_size}, time: {time}s, n_channels: {n_channels}")


sample_rate: 64, n_samples = 640, time: 10.0s, n_channels: 7


In [24]:
held_out, groups, group_assignments = get_group_assignments(
    df)
n_groups = len(groups)
# cross validation splits    
group_kfold = GroupKFold(n_splits=n_groups)

print(f"len(groups) = {len(groups)}")
print(f"len(group_assignments) = {len(group_assignments)}")


len(groups) = 11
len(group_assignments) = 9906


In [25]:
feature_types, feature_names, designmtx = get_design_matrix(
    df, feature_set)
# extract labels
labels = df['condition'].values.astype(int)
X = designmtx
y = labels

print(f"X.shape = {X.shape}")
print(f"y.shape = {y.shape}")


X.shape = (9906, 159)
y.shape = (9906,)


In [33]:
for i, (train_index, test_index) in enumerate(group_kfold.split(X, y, group_assignments)):

    print(f"Fold {i}:")
#     print(f"  Train:\n\tindex={train_index},\n\tgroup={group_assignments[train_index]}")
    print(f"  Test:")
#     print(f"\tindex={test_index},")
    print(f"\tgroup={np.unique(group_assignments[test_index])}")

Fold 0:
  Test:
	group=[0]
Fold 1:
  Test:
	group=[10]
Fold 2:
  Test:
	group=[9]
Fold 3:
  Test:
	group=[3]
Fold 4:
  Test:
	group=[2]
Fold 5:
  Test:
	group=[6]
Fold 6:
  Test:
	group=[5]
Fold 7:
  Test:
	group=[7]
Fold 8:
  Test:
	group=[1]
Fold 9:
  Test:
	group=[8]
Fold 10:
  Test:
	group=[4]


In [36]:
test_participants = []
for i, (train_index, test_index) in enumerate(group_kfold.split(X, y, group_assignments)):
    fold_pids = np.unique(group_assignments[test_index])
    fold_participants = [ participant_list[p] for p in fold_pids]
    test_participants.append(fold_participants)
test_participants

[['VG_01'],
 ['VH_03'],
 ['VH_02'],
 ['VG_06'],
 ['VG_05'],
 ['VG_09'],
 ['VG_08'],
 ['VG_10'],
 ['VG_03'],
 ['VH_01'],
 ['VG_07']]